## Experiment 1: (2022-07-28~)
- - -
#### 0. Preprocessing / (Done on seperate file)
Urbansound8k Dataset i){8732(<=4s).wav files/10 folders(must be separated)} => Noise data    
MTG-jamendo Dataset i){}=> Music data   
For easy labeling, we need to save the labels along with the data ( X.npy = [ data , label ] )

#### 1. Initialize input pipeline


#### 2. Model initialization


#### 3. Training & Evaluation

- - -

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses

import tensorflow_addons as tfa

import pathlib

import librosa

import soundfile as sf

from tqdm import tqdm

from IPython.display import clear_output

autotune = tf.data.AUTOTUNE

In [35]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            tf.config.experimental.set_virtual_device_configuration(gpu, [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*11)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

Physical devices cannot be modified after being initialized


## 1. Initializing Input Pipeline   
(planned 22.08.08)   
* First define the raw tf.data.Datasets.   
* Make a function that preprocesses the training datasets.
* Use tf.data.Dataset.map to create the fully processed dataset.
* Then make a function that finds the labels for each of the training datasets.   
* Use tf.data.Dataset.map to create the label Dataset.   
* Then combine them with tf.data.Dataset.zip.   
* Then Shuffle & Configure for performance

In [2]:
IMAGE_SIZE = 513*345
IMAGE_DIM = [513,345,1]

# Training
batch_size = 1

In [3]:
base_path = pathlib.Path('D:/전재원졸업연구/music-discern/') # Edit on other computers
base_path_dataset = pathlib.Path('D:/전재원졸업연구/music-discern/Datasets/') # Edit on other computers

#Urbansound is nested in Datasets/Urbansound8K/audio/fold**
#Urbansound is already separated into batches, which need to be preserved.
urban_metadata_path = pathlib.Path(base_path_dataset/'Urbansound8K/metadata')
urban_audio_folders_path = pathlib.Path(base_path_dataset/'Urbansound8K/processed')
urban_audio_path_list = [x for x in urban_audio_folders_path.iterdir()]
temp = urban_audio_path_list.pop(1)
urban_audio_path_list.append(temp)

#Jamendo is nested in Datasets/MTGJamendo/audio
#Jamendo is not separated in batches, so we need to separate it into 10 batches, matching urbansound.
jamendo_metadata_path = pathlib.Path(base_path_dataset/'MTGJamendo/metadata')
jamendo_audio_folders_path = pathlib.Path(base_path_dataset/'MTGJamendo/processed')
jamendo_audio_path_list = [x for x in jamendo_audio_folders_path.iterdir()]
temp = jamendo_audio_path_list.pop(1)
jamendo_audio_path_list.append(temp)


#for later -> add a path for any .wav file that I want to observe.
test_subjects_path = pathlib.Path("./")

save_weights_path = pathlib.Path(base_path/'Experiment1/model_checkpoints')
save_weights_A_path = pathlib.Path(save_weights_path/'model_A')
save_weights_B_path = pathlib.Path(save_weights_path/'model_B')

Train = [0,1,2,3,4,5,6,7,8]
Test = [9] #Seperation, for later evaluation in Training step B

print(urban_audio_path_list)
print(jamendo_audio_path_list)

[WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold1'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold2'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold3'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold4'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold5'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold6'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold7'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold8'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold9'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold10')]
[WindowsPath('D:/전재원졸업연구/music-discern/Datasets/MTGJamendo/processed/fold1'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/MTGJamendo/processed/fold2'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/MTGJam

In [4]:
dataset_for_training_A = "empty"
dataset_for_training_B_train = "empty"
dataset_for_training_B_test = "empty"

dataset_for_training_A = tf.data.Dataset.list_files(str(urban_audio_folders_path/'*/*.npy'))
dataset_for_training_A = dataset_for_training_A.concatenate( tf.data.Dataset.list_files(str(jamendo_audio_folders_path/'*/*.npy')) )

for k in Train:
    temp_path_urban = str(urban_audio_path_list[k])
    temp_path_jam = str(jamendo_audio_path_list[k])
    if dataset_for_training_B_train == "empty":
        dataset_for_training_B_train = tf.data.Dataset.list_files(temp_path_urban + '/*.npy')
        dataset_for_training_B_train = dataset_for_training_B_train.concatenate( tf.data.Dataset.list_files(temp_path_jam + '/*.npy') )
    else:
        dataset_for_training_B_train = dataset_for_training_B_train.concatenate( tf.data.Dataset.list_files(temp_path_urban + '/*.npy') )
        dataset_for_training_B_train = dataset_for_training_B_train.concatenate( tf.data.Dataset.list_files(temp_path_jam + '/*.npy') )

for k in Test:
    temp_path_urban = str(urban_audio_path_list[k])
    temp_path_jam = str(jamendo_audio_path_list[k])
    if dataset_for_training_B_test == "empty":
        dataset_for_training_B_test = tf.data.Dataset.list_files(temp_path_urban + '/*.npy')
        dataset_for_training_B_test = dataset_for_training_B_test.concatenate( tf.data.Dataset.list_files(temp_path_jam + '/*.npy') )
    else:
        dataset_for_training_B_test = dataset_for_training_B_test.concatenate( tf.data.Dataset.list_files(temp_path_urban + '/*.npy') )
        dataset_for_training_B_test = dataset_for_training_B_test.concatenate( tf.data.Dataset.list_files(temp_path_jam + '/*.npy') )

# returns dataset for training A, dataset for training B Train, dataset for training B Test

In [5]:
for f in dataset_for_training_A.take(1):
    print(f)
    print(tf.strings.split( tf.strings.split( f,os.sep )[-1],sep="_" )[1])
    A = np.load(f.numpy(),allow_pickle=True)
    print(A.dtype)
    img = tf.io.read_file(f)
    img = tf.io.decode_raw(img,tf.complex64)
    img = tf.slice(img,begin=[16],size=[IMAGE_SIZE])
    img = tf.reshape(img,[513,345])
    print(img)

tf.Tensor(b'D:\\\xec\xa0\x84\xec\x9e\xac\xec\x9b\x90\xec\xa1\xb8\xec\x97\x85\xec\x97\xb0\xea\xb5\xac\\music-discern\\Datasets\\Urbansound8K\\processed\\fold9\\159748-8-2-1_noise_0.npy', shape=(), dtype=string)
tf.Tensor(b'noise', shape=(), dtype=string)
complex64
tf.Tensor(
[[-7.4058610e-01+0.0000000e+00j  7.2538865e-01+1.9831434e-03j
  -9.6392304e-01-3.0947492e-02j ... -5.3103864e-03+1.2901510e-02j
   3.1638551e-03-1.0513442e-02j -4.0921038e-03+8.4731560e-03j]
 [ 4.8934552e-03-8.1008328e-03j -5.5898461e-03+7.2627570e-03j
   7.4535613e-03-7.4173650e-03j ... -7.5784479e-03-2.1927562e-02j
   1.0540253e-04-4.5235227e-03j -3.1723292e-03+2.3954416e-02j]
 [ 1.4081898e-02-4.0049204e-03j -1.0653642e-02-5.2852076e-03j
   2.4423981e-03-1.0678490e-02j ... -2.2904472e+00+5.2099156e+00j
  -6.8144083e-01-5.9907241e+00j  2.2153766e+00+2.5216300e+00j]
 ...
 [-2.5041163e-07-7.8813355e-07j  1.7617194e-06+3.6402641e-06j
   2.1978199e-06-2.9133366e-06j ... -6.9954935e-05-3.2876257e-04j
   1.1841396e-03+6.

In [6]:
def process_image(image):
    #make the images into spectrograms
    processed_img = tf.pow(tf.abs(image),2)
    #normalize the images?
    return processed_img

def process_path_A(path):
    #for training A, the label is just itself so we need to take that into account
    img = tf.io.read_file(path)
    img = tf.io.decode_raw(img,tf.complex64)
    img = tf.slice(img,begin=[1024//64],size=[IMAGE_SIZE]) # begin <- 1024//complex64
    img = tf.reshape(img,IMAGE_DIM)
    img = process_image(img)

    label = img
    return img, label

def process_path_B(path): #map_func
    #extract the 2d array and the label
    img = tf.io.read_file(path)
    img = tf.io.decode_raw(img,tf.complex64)
    img = tf.slice(img,begin=[1024//64],size=[IMAGE_SIZE]) # begin <- 1024//complex64
    img = tf.reshape(img,IMAGE_DIM)
    img = process_image(img)

    label = tf.strings.split( tf.strings.split( path, os.sep )[-1],sep="_" )[1]
    return img, label

In [7]:
dataset_A = dataset_for_training_A.map(process_path_A,num_parallel_calls=autotune)
dataset_B_train = dataset_for_training_B_train.map(process_path_B,num_parallel_calls=autotune)
dataset_B_test = dataset_for_training_B_test.map(process_path_B,num_parallel_calls=autotune)

In [58]:
next(iter(dataset_A))[1]

<tf.Tensor: shape=(1, 513, 345, 1), dtype=float32, numpy=
array([[[[2.75533814e-02],
         [1.87707860e-02],
         [5.02258688e-02],
         ...,
         [1.55973034e-02],
         [1.13079082e-02],
         [1.30104311e-02]],

        [[7.54628424e-03],
         [6.75441464e-03],
         [2.05090512e-02],
         ...,
         [2.13153648e+00],
         [1.33940477e+01],
         [1.84182453e+01]],

        [[9.00236320e+00],
         [2.50321555e+00],
         [1.69781017e+00],
         ...,
         [1.41896531e-02],
         [2.17648089e-01],
         [6.53459579e-02]],

        ...,

        [[1.03589414e-07],
         [1.24638149e-08],
         [2.48422822e-07],
         ...,
         [1.04125746e-01],
         [5.46079986e-02],
         [3.00682522e-02]],

        [[2.91865841e-02],
         [2.73851361e-02],
         [2.99190637e-02],
         ...,
         [5.82082415e+00],
         [1.19604290e+00],
         [1.00237017e+01]],

        [[2.02889252e+01],
         [3

In [9]:
def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=autotune)
    return ds

dataset_A = configure_for_performance(dataset_A)
dataset_B_test = configure_for_performance(dataset_B_test)
dataset_B_train = configure_for_performance(dataset_B_train)

In [174]:

a = layers.Conv2D(16, (3,3), activation='relu', padding='same', strides=2)
b = layers.Conv2D(8, (3,3), activation='relu', padding='same', strides=2)
c = layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same')
d = layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same')
e = layers.Conv2D(1, kernel_size=(3,3), activation='sigmoid', padding='same')

input_shape = (1,513,345,1)
x = tf.random.normal(input_shape)

print(input_shape)
y = a(x)
print(y.shape)
y = b(y)
print(y.shape)
y = c(y)
print(y.shape)
y = d(y)
print(y.shape)
y = e(y)
print(y.shape)
y = e(d(c(b(a(x)))))
print(y.shape)

(1, 513, 345, 1)
(1, 257, 173, 16)
(1, 129, 87, 8)
(1, 258, 174, 8)
(1, 516, 348, 16)
(1, 516, 348, 1)
(1, 516, 348, 1)


## 2. Model initialization

In [59]:
class Autoencoder(tf.keras.Model):

    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            layers.InputLayer(input_shape = IMAGE_DIM , batch_size= batch_size),
            layers.Conv2D(16, (3,3), activation='relu', padding='same', strides=2),
            layers.Conv2D(8, (3,3), activation='relu', padding='same', strides=2),
        ])
        self.decoder = tf.keras.Sequential([
            layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'),
            layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
            layers.Conv2D(1, kernel_size=(3,3), activation='sigmoid', padding='same')
        ])
    
    def __call__(self, x, training=False):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
Model_A = Autoencoder()

In [12]:
class Classifier(tf.keras.Model):

    def __init__(self, Model_A):
        super(Classifier, self).__init__()
        #comes after the encoding
        self.classifer_layer = tf.keras.Sequential([
            layers.InputLayer(input_shape=[129,87,8], batch_size= batch_size),
            layers.Conv2D(8, (3,3), activation='relu', padding='same', strides=2),
            layers.Conv2D(4, (3,3), activation='relu', padding='same', strides=2),
            layers.Conv2D(2, (3,3), activation='relu', padding='same', strides=2),
            layers.Conv2D(1, (3,3), activation='relu', padding='same', strides=2),
            layers.Flatten(),
            layers.Dense(16, activation='relu'),
            layers.Dense(1, activation='softmax')
        ])
        self.Model_A = Model_A.encoder # Probablility of deepcopy, not sure. If accuracy is terrible this may be the suspect

    def __call__(self,x):
        compressed = self.Model_A(x)
        return self.classifer_layer(compressed)

Model_B = Classifier(Model_A)

In [109]:
Model_A.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss=tf.keras.losses.MeanSquaredError())

In [14]:
Model_B.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [61]:
# Create a callback function that saves the weights periodically, and shows the reconstructed images periodically
class Monitor_A(keras.callbacks.Callback):
    
    def on_epoch_end(self, epoch, logs=None):
        _, ax = plt.subplots(1,2, figsize=(21,7))
        for k,k in dataset_A.take(1):
            changed = self.model(k)
            clear_output(wait=True)
            ax[0, 0].imshow(k)
            ax[0, 1].imshow(changed)
            ax[0, 0].set_title("Input image")
            ax[0, 1].set_title("Translated image")
            ax[0, 0].axis("off")
            ax[0, 1].axis("off")
        
        if (epoch) % 5 == 0:
            self.model.save_weights(pathlib.Path(save_weights_A_path/"{epoch:04}.h5"))
    
        plt.show()
        plt.close()

In [16]:
# Create a callback function that saves the weights periodically, and shows the reconstructed images periodically
class Monitor_B(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        
        if (epoch) % 5 == 0:
            self.model.save_weights(pathlib.Path(save_weights_B_path/"{epoch:04}.h5"))


## 3. Training & Evaluation

In [168]:
with tf.device('/cpu:0'):
    history = Model_A.fit(dataset_A, batch_size=batch_size, epochs=100, callbacks=[Monitor_A()])

Epoch 1/100


ValueError: in user code:

    c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\keras\engine\training.py:855 train_function  *
        return step_function(self, iterator)
    c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\keras\engine\training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3608 _call_for_each_replica
   

In [93]:
Model_A.fit(dataset_A, batch_size=batch_size, epochs=100, callbacks=[Monitor_A()])

Epoch 1/100


TypeError: 'NoneType' object is not callable

In [ ]:
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()

In [ ]:
with tf.device('/gpu:0'):
    history = Model_B.fit(dataset_B_train, batch_size=batch_size, epochs=100, callbacks=[Monitor_A], validation_data=dataset_B_test)

![](/Experiment1abstract.jpg?raw=true)